# Generate stories using LLMs through API inference

In [ ]:
import os
import json
from openai import OpenAI
from together import Together

In [ ]:
# Param
results_base_path = "../../results"

dataset_name = "stories"
#dataset_name = "stories_val"  # Validation dataset

experiment_name = "exp1"

# Generator LLM
gen_model = "gpt-4o-mini"
#gen_model = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
#gen_model = "google/gemma-2-27b-it"
#gen_model = "mistralai/Mistral-7B-Instruct-v0.2"
#gen_model = "Qwen/Qwen2.5-7B-Instruct-Turbo"
#gen_model = "claude-3-7-sonnet-20250219"  # Validation dataset

system_prompt = None

dataset_base_path = "../../data"

#lang = "cs"
lang = "sl"

In [ ]:
if gen_model.startswith("gpt"):
    client = OpenAI()
elif gen_model.startswith("claude"):
    client = OpenAI(
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
        base_url="https://api.anthropic.com/v1/",
    )
else:
    client = Together()

def inference(client, messages):
    response = client.chat.completions.create(model=gen_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
def add_conversation_round(conversation, new_prompt):
    conversation.append({"role": "user", "content": new_prompt})
    response = inference(client, conversation)
    conversation.append({"role": "assistant", "content": response})
    return conversation

In [ ]:
# Load the dataset from local JSON
with open(os.path.join(dataset_base_path, dataset_name, f"{lang}.json"), "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [ ]:
# Create the directory structure
os.makedirs(os.path.join(results_base_path, dataset_name, experiment_name, gen_model, lang), exist_ok=True)

# Iterate through prompts
for id, sample in enumerate(dataset):
    prompt = sample["prompt"]

    # Initialize conversation history
    if system_prompt:
        conversation = [{"role": "system", "content": system_prompt}]
    else:
        conversation = []

    # Add the prompt to the conversation and get the response
    conversation = add_conversation_round(conversation, prompt)

    # Prepare result
    result = {
        "id": id,
        "conversation": conversation
    }

    # Save to JSON
    output_file = os.path.join(results_base_path, dataset_name, experiment_name, gen_model, lang, f"{id:06d}.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=4, ensure_ascii=False)

    # Print progress
    print("id:", id)
    print("Prompt:", prompt)
    print("Response:", conversation[-1]["content"])
    print("-" * 50)